# Plotting: particles

In this notebook, we will load a cosmological simulation and visualize the dark matter particles within it.

## A basic look at the particle positions

In [ ]:
import osyris

path = "osyrisdata/cosmology"
data = osyris.RamsesDataset(100, path=path).load("part")

We convert the particle masses to solar masses, and their positions to megaparsecs:

In [ ]:
part = data["part"]

part["mass"] = part["mass"].to("M_sun")
part["position"] = part["position"].to("Mpc")
part

Having loaded the particle data,
we can quickly inspect the distribution of the particles by histogramming their `x` and `y` positions.
We use the particle mass as weights for the histogramming,
to give us a feel for how much mass is contained in one of the pixels of the image below.

In [ ]:
osyris.hist2d(
    part["position"].x,
    part["position"].y,
    part["mass"],
    bins=512,
    norm="log",
)

## Sorting particles

When particles are loaded, they are read in in the order they are stored on disk.
It can however often be useful to sort particles according to one of the keys.
A common choice is to sort them according to their `identity` using the `.sortby()` method of the `Datagroup`.

In [ ]:
print(part["identity"][:20].values)
part.sortby("identity")
print(part["identity"][:20].values)

It is also possible to apply sorting by using a keyword argument in the `load()` function:

In [ ]:
data = osyris.RamsesDataset(100, path=path).load("part", sortby={"part": "identity"})
print(data["part"]["identity"][:20].values)

## 3D rendering

Osyris does not support 3D plotting out of the box,
but a 3D rendering of the particle positions in space can be achieved relatively easily in a Jupyter notebook using the `pythreejs` library
(or any other library with 3D capabilities).

<div class="alert alert-info">

**Note**

`pythreejs` is not a hard-dependency of Osyris. It needs to be installed separatey with `pip install pythreejs`.

</div>

In [ ]:
import pythreejs as p3
import numpy as np

# Only plot one in 10 points to reduce the size of the output
pos = part["position"][::10]
pos_array = np.array([pos.x.values, pos.y.values, pos.z.values]).T.astype("float32")

# Create a position buffer geometry
geometry = p3.BufferGeometry(
    attributes={"position": p3.BufferAttribute(array=pos_array - 75)}
)
# Create a points material
material = p3.PointsMaterial(color="black", size=1)
# Combine the geometry and material into a Points object
points = p3.Points(geometry=geometry, material=material)

# Create the scene and the renderer
view_width = 700
view_height = 500
camera = p3.PerspectiveCamera(position=[200.0, 0, 0], aspect=view_width / view_height)
scene = p3.Scene(children=[points, camera], background="#DDDDDD")
controller = p3.OrbitControls(controlling=camera)
renderer = p3.Renderer(
    camera=camera,
    scene=scene,
    controls=[controller],
    width=view_width,
    height=view_height,
)
renderer